## Demonstration of when the Pandas SettingWithCopyWarning occurs and how to avoid it

In [1]:
import numpy as np
import pandas as pd
pd.__version__

'2.0.3'

In [2]:
# Create some random data
data = {
    'A': [1, 2, 3],
    'B': [0.1, 0.2, 0.4],
    'C': ['x', 'y', 'z']
}

# Make a Pandas dataframe
df = pd.DataFrame(data)
df

,A,B,C
0,1,0.1,x
1,2,0.2,y
2,3,0.4,z


In [3]:
# Take a slice of the dataframe
first = df.loc[0, :]
first

A      1
B    0.1
C      x
Name: 0, dtype: object

In [4]:
# Try to change one of the values in the slice
first['B'] = 99

/var/folders/y3/28pc8qrx3dd36zwcys8z1rzc0000gn/T/ipykernel_15826/1080533129.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first['B'] = 99


In [5]:
# Even though there was a warning, it did work...
first

A     1
B    99
C     x
Name: 0, dtype: object

In [6]:
# But the original dataframe was not affected
df

,A,B,C
0,1,0.1,x
1,2,0.2,y
2,3,0.4,z


In [7]:
# Just to verify that the slice is a copy, let's change
# a value in the original dataframe
df.loc[0,'C'] = "XXX"

In [8]:
df

,A,B,C
0,1,0.1,XXX
1,2,0.2,y
2,3,0.4,z


In [9]:
first
# It must be a copy

A     1
B    99
C     x
Name: 0, dtype: object

In [10]:
# You can use this private attribute to check
first._is_view

False

In [11]:
# To avoid the SettingWithCopyWarning, just make the fact that
# you are making a copy explicit
second = df.loc[1,:].copy()

In [12]:
second['B'] = 9.99
# No warning was raised!

In [13]:
second

A       2
B    9.99
C       y
Name: 1, dtype: object

## Conclusion

The details of when Pandas makes a copy or gives you a view are not clear. [See this discussion](https://stackoverflow.com/questions/23296282/what-rules-does-pandas-use-to-generate-a-view-vs-a-copy) and the examples below. They are an implementation detail intended to minimize memory usage and computation.

Therefore, **you should never rely on the fact that you have a view or a copy in your code**. The only time it really matters is when you make changes to your view/copy. If you do not want to make changes to the original dataframe, you make sure it's a copy by using the copy method, as in the example above.

If you do want to make changes to the original dataframe, you should explicitely write those changes to the original dataframe.  For example:

In [14]:
# Instead of first['B'] = 9.99, use the loc method on the original dataframe
df.loc[0, 'B'] = 9.99
df

,A,B,C
0,1,9.99,XXX
1,2,0.20,y
2,3,0.40,z


# Mysteries of copy/view

In [15]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
df

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [16]:
df['A']._is_view

True

In [17]:
df[['A', 'B']]._is_view

False

In [18]:
df.iloc[:, 0:2]._is_view

True

In [19]:
df.loc[0]._is_view

True

In [20]:
df.loc[[0, 1]]._is_view

False

In [21]:
df.loc[0:2]._is_view

True

In [22]:
df.loc[0]._is_view

True

In [23]:
df.loc[0:2, 'A']._is_view

True

In [24]:
df.loc[0:2, ['A', 'B']]._is_view

False

In [25]:
df.iloc[0:2, 0:2]._is_view

True

In [26]:
data = np.array([[1, 4, 7], [2, 5, 8], [3, 6, 9]])
df = pd.DataFrame(data, columns=['A', 'B', 'C'])
df

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [27]:
df['A']._is_view

True

In [28]:
df[['A', 'B']]._is_view

True

In [29]:
df.iloc[:, 0:2]._is_view

True

In [30]:
df.loc[0]._is_view

True

In [31]:
df.loc[[0, 1]]._is_view

True

In [32]:
df.loc[0:2]._is_view

True

In [33]:
df.loc[0]._is_view

True

In [34]:
df.loc[0:2, 'A']._is_view

True

In [35]:
df.loc[0:2, ['A', 'B']]._is_view

True

In [36]:
df.iloc[0:2, 0:2]._is_view

True

In [37]:
df2 = pd.concat([df.copy(), df.copy().rename(columns={'A': 'C', 'B': 'D', 'C': 'E'})], axis=1)
df2

,A,B,C,C,D,E
0,1,4,7,1,4,7
1,2,5,8,2,5,8
2,3,6,9,3,6,9


In [38]:
df2['A']._is_view

True

In [39]:
df2[['A', 'B']]._is_view

False

In [40]:
df2.loc[0]._is_view

False

In [41]:
df2.iloc[:, 0]._is_view

True

In [42]:
df2.iloc[0, :]._is_view

False

In [43]:
df2.iloc[:, 0:2]._is_view

False

In [44]:
df2.iloc[:, 3:5]._is_view

False